In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import re

In [2]:
faq = pd.read_csv('./faqlist_0127.csv', low_memory=False)
#faq = pd.read_csv('./공지2.csv', low_memory=False)

In [3]:
#faq.head(10)
#print(faq)
faq

,idx,title,answer
0,0,[학술정보지원팀] <자료구입신청>\n자료 구입 신청은 어떻게 하나요?,"[서울캠퍼스]\n우리 대학에 소속된 학부생, 대학원생, 교직원이라면 누구든지 이용하..."
1,1,[학술정보지원팀] <자료구입신청>\n신청한 책이 취소되었습니다. 왜 취소된 건가요?,[서울캠퍼스]\n학술정보관은 신청되는 모든 자료를 입수하기 위해 노력하고 있습니다....
2,2,[학술정보지원팀] <자료구입신청>\n신청한 도서의 진행 상태는 어떻게 알 수 있나요?,[서울캠퍼스]\n학술정보관 홈페이지를 통해 확인할 수 있습니다.\n신청도서가 ‘대출...
3,3,[학술정보지원팀] <자료기증>\n도서를 기증하고 싶습니다. 어떻게 해야 하나요?,[서울캠퍼스]\n도서를 기증하시는 방법은 아래의 3가지가 있습니다.\n1. 직접기증...
4,4,[학술정보지원팀] <학술정보관 이용>\n신입생은 학술정보관을 어떻게 이용하나요?,[서울캠퍼스]\n-[서울캠퍼스]\n\n[천안캠퍼스]\n신입생의 경우 학생증 발급 이...
...,...,...,...
379,379,[정보통신팀] 학생메일 신청은 어떻게 하나요?,교내 포털시스템에 접속 후 오른쪽 퀵메뉴에 OFFICE 365 버튼을 클릭하시고 비...
380,380,[정보통신팀] 학생메일의 비밀번호를 잃어버렸습니다.,office365의 비밀번호는 포털 시스템과 동일하게 운영됩니다. 포털 비밀번호를 ...
381,381,[정보통신팀] 교직원 메일의 비밀번호를 잃어버렸습니다.,1. 교내 포털시스템 접속 후 오른쪽 상단에 내정보 클릭\n2. 개인정보변경 클릭\...
382,382,[정보통신팀] 통합정보비밀번호를 잃어버렸습니다.,1. https://portal.smu.ac.kr 의 로그인 페이지에서 비밀번호 초...


In [10]:
faq.answer[0]

'[서울캠퍼스]\n우리 대학에 소속된 학부생, 대학원생, 교직원이라면 누구든지 이용하고 싶은 자료의 구입을 신청할 수 있습니다. 도서, 비도서 뿐만 아니라 e-book도 신청할 수 있습니다.\n신청된 자료는 학술정보관 자료선정기준에 의거하여 예산 범위내에서 구입하도록 하겠습니다. 유의사항을 통해 신청이 제한되는 자료를 확인하여 주시기 바랍니다.\n개인정보에 휴대전화번호를 등록하였을 경우, 문자메시지를 통해 희망도서 도착통보를 받을 수 있습니다. 자료가 악보, 음악 CD/DVD 등 음악과 관련된 자료일 경우에는 음대자료실 대출규정에 따라 이용이 가능합니다.\n\n<유의사항>\n1. 신청가능 종수와 금액\n 가. 종수: 1인 기준 주당 5종, 학기당 30종\n 나. 금액: 1종당 10만원, 1인 기준 학기당 50만원\n2. 신청가능 신분 : 상명대학교 서울캠퍼스 소속 재학생과 교직원(※휴학생, 졸업생, 수료생, 지역주민 불가)\n3. 구입불가자료\n 가. 기(旣) 소장 도서 또는 주문중인 도서\n 나. 만화류\n 다. 유아·아동물\n 라. 초·중·고 교과서 및 참고서\n 마. 노골적인 성인물\n 바. 사설학원 교재 등 특정 개인에게만 필요한 도서\n 사. 규격외 문제집(8절, 낱장, 스프링 제본) 등 관리가 어려운 도서\n 아. 품절·절판 등으로 구매가 불가능한 경우\n 자. 장르문학: 무협지, 판타지, 로맨스 등(라이트 노벨 포함)\n 차. 정기간행물: 잡지, 학회지, 논문집 등\n 카. 기타 대학도서관 소장 자료로서 부적절한 도서[서울캠퍼스]\n\n[천안캠퍼스]\n학술정보관 홈페이지에서 로그인(재학생/교직원) 한 후 자료구입신청할 수 있으며 도서 뿐만 아니라 dvd, e-book 등도 신청가능합니다. \n신청방법은 학술정보관 홈페이지(http://libnt.smuc.ac.kr) → 로그인 → 자료구입신청에서 신청 가능하며 상세 자료구입신청 방법은 홈페이지를 참고하시기 바랍니다.'

In [4]:
# 코사인 유사도 

def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

In [5]:
from konlpy.tag import Okt
okt = Okt()
stopwords = ['은','는','이','가']

In [6]:
from pykospacing import spacing # 띄어쓰기 교정
from hanspell import spell_checker # 맞춤법 교정

In [7]:
temp = pd.DataFrame(columns=['title'])
temp['title'] = faq['title'].copy()
temp['title'] = temp['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-zA-Z]"," ")

for i in range(len(temp)):   
    temp['title'][i] = temp['title'][i].lower()
    temp['title'][i] = spacing(temp['title'][i]) #띄어쓰기 교정
    temp['title'][i] = spell_checker.check(temp['title'][i]).checked # 맞춤법 교정
    
    temp['title'][i] = okt.morphs(temp['title'][i], stem=True)
    temp['title'][i] = [word for word in temp['title'][i] if not word in stopwords]
faq['title2'] = temp['title'].copy()

In [36]:
# 챗봇 사용자 채팅

usr_question = input("사용자 질문 : ")

사용자 질문 :  학점포기 기간 알려줘


In [37]:
### 사용자 질문 처리

question = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 a-zA-Z]", " ", usr_question)
question = question.lower()
question = spacing(question)
question = spell_checker.check(question).checked
question = okt.morphs(question, stem=True)
question = [word for word in question if not word in stopwords]

print(question)   

['학점', '포기', '기간', '알다']


In [38]:
question = ' '.join(question)
temp_list = [question]

for i in range(len(faq['title2'])):
    join_str = ' '.join(faq['title2'][i])
    temp_list.append(join_str)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(temp_list)
#print(feature_vect_simple.shape)
#print(type(feature_vect_simple))

# TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환.
feature_vect_dense = feature_vect_simple.todense()

#첫번째 문장과 두번째 문장의 feature vector  추출
#vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
#vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

#첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
#similarity_simple = cos_similarity(vect1, vect2)
#print('문장 1, 문장 2 Cosine 유사도: {0:.3f}'.format(similarity_simple))

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

#similarity_simple_pair = cosine_similarity(feature_vect_simple[0] , feature_vect_simple)
#print(similarity_simple_pair)

similarity_simple_pair = cosine_similarity(feature_vect_simple , feature_vect_simple)
#print(similarity_simple_pair)
#print(similarity_simple_pair[0][1:])
result = similarity_simple_pair[0][1:]
print(result)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.10926752 0.         0.         0.         0.
 0.         0.         0.         0.17639724 0.         0.18840271
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.18840271
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.16200637 0.
 0.         0.17639724 0.         0.18840271 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.    

In [41]:
val_index = np.argmax(result)
result_dic = dict(enumerate(result))
res = sorted(result_dic.items(), key=(lambda x: x[1]), reverse = True)

In [42]:
print("질문 : " + usr_question)
#print("질문(필터링) : " + question + "\n")
print("\n[유사도 기반 FAQ List Top3]\n")

sum = 0
seq = 1
for k, v in res[:3]:
    
    if v != 0:
        print(seq, ".", faq['title'][k])
        print(faq['answer'][k])
        print("\n")
        sum = sum+1
        seq = seq+1
        

if sum == 0:
    print("검색결과가 없습니다.")
    print("필수 단어 위주로 검색해보세요.")
    
else:
    #print('\n')
    print("※찾고자하는 답변이 아니신가요? 그렇다면 필수 단어 위주로 검색해보세요.")

질문 : 학점포기 기간 알려줘

[유사도 기반 FAQ List Top3]

1 . [학점인정] 학점포기 신청기간이 어떻게 되나요?
매학기 마다 공지사항이 게시되며, 특정 기간내에만 신청 가능합니다. 학점포기신청은 샘물통합정보시스템→학생기본→성적정보→학점포기신청→학년도 학기선택→ 희망과목 포기신청→ 일괄확정(최종완료)로 진행되며 [성적정보→전체성적조회]에서 성적반영 여부를 반드시 확인바랍니다.


2 . [학점인정] 학점포기 대상이 어떻게 되나요?
학점포기 제도는 2013학년도 이전 입학한 4학년 이상 재학생(9학기 이상 재학생 포함)만 신청이 가능하며, 2013년도 이후 수강한 수업은 포기 신청이 불가합니다. 


3 . [학점인정] 학점포기한 학점을 다시 복원할 수 있나요?
아니요. 학점포기한 학점은 절대 복원되지 않습니다. 이에, 신중하게 신청을 해주시기 바랍니다.


※찾고자하는 답변이 아니신가요? 그렇다면 필수 단어 위주로 검색해보세요.
